In [5]:
from utils.audio_dataset_utils import train_val_dataset
from network_models.soundsream_models_and_utils.ss_model_dim_red import SSDimRedModel
from network_models.soundsream_models_and_utils.ss_model_conv import SSConvModel3Sec
from network_models.soundsream_models_and_utils.ss_encoded_dataset import ss_encoded_dataset_full
import torch
from torch.utils.data import DataLoader
from network_models.soundsream_models_and_utils.ss_complex_conv_net import SSComplexConvModel3Sec

batch_size = 6

data_set= ss_encoded_dataset_full(
    csvPath="/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/data/allEncs_betterSS_12_1_relwrap_music_tess.pkl", device="cuda")


modelDimRed = SSDimRedModel(eval_mode= True).cuda()
modelConv = SSConvModel3Sec(xSize=512, ySize=175, eval_mode= True).cuda()
modelComplexConv = SSComplexConvModel3Sec(eval_mode=True).to("cuda")

trainDS, testDs = train_val_dataset(data_set, val_split=0.1, seed=100)
trainDS, valDs = train_val_dataset(trainDS, val_split=0.1, seed=100)

In [6]:
import gc

gc.collect()

#modelConv.load_state_dict(torch.load("../content/soundstream/experiments/experiment_v_12_1_NR3_relu/Run_Nr_0/conv/emo_reco_best_ep852_acc_75.pth"))
#modelDimRed.load_state_dict(torch.load("../content/soundstream/experiments/experiment_v_12_1_NR3_relu/Run_Nr_0/dimred/emo_reco_best_ep824_acc_70.pth"))
modelDimRed.load_state_dict(torch.load("/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/soundstream/experiments/dim_red/Nr1_tess/emo_reco_best_ep105_acc_96.pth"))
#modelComplexConv.load_state_dict(torch.load("/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/soundstream/experiments/conv_complex/Nr2_tess/emo_reco_best_ep92_acc_100.pth"))
#modelComplexConv.load_state_dict(torch.load("/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/soundstream/experiments/conv_complex/Nr4/emo_reco_best_ep95_acc_69.pth"))
#modelComplexConv.load_state_dict(torch.load("/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/soundstream/experiments/conv_complex/Nr2_tess/emo_reco_best_ep92_acc_100.pth"))


<All keys matched successfully>

In [7]:
import gc


def evaluate(device, model, dataset):
    loader = DataLoader(dataset, shuffle=False, batch_size=batch_size)
    true, preds = [], []
    with torch.no_grad():
        for batch, (X, z) in enumerate(loader):
            X, z = X.to(device),  z.to(device)
            pred = model(X)
            labels = [torch.squeeze(a.nonzero()).item() for a in z]
            true = true + labels

            preds = preds + pred.argmax(1).cpu().numpy().tolist()
            #preds.append(pred.argmax(1))
            if batch % 10 == 0:
                print(f"{batch*batch_size} of {len(dataset)}")

    return true, preds

In [13]:
from utils.eval_utils import classificationReport, confusion_matrix
trueConv, predsConv = evaluate("cuda", modelDimRed, testDs)
print("------------------ Conv: ----------------")
classificationReport(trueConv, predsConv, data_set.encoded_dataset.label_list)
confusion_matrix(trueConv, predsConv, data_set.encoded_dataset.label_list)

print()

0 of 276
60 of 276
120 of 276
180 of 276
240 of 276
------------------ Conv: ----------------
           accuracy  precision  recall   support
   angry     0.931     1.000     0.977    44
 disgust     0.931     0.923     0.667    36
    fear     0.931     0.952     1.000    40
   happy     0.931     0.896     0.935    46
 neutral     0.931     1.000     1.000    36
     sad     0.931     0.971     0.944    36
surprise     0.931     0.804     0.974    38
                                          276

 
     avg     0.931     0.935     0.928    

                   angry     disgust        fear       happy     neutral         sad    surprise    support
       angry       0.977       0.000       0.023       0.000       0.000       0.000       0.000         44
     disgust       0.000       0.667       0.000       0.111       0.000       0.028       0.194         36
        fear       0.000       0.000       1.000       0.000       0.000       0.000       0.000         40
       happy     

In [14]:
from utils.eval_utils import classificationReport, confusion_matrix

trueConv, predsConv = evaluate("cuda", modelConv, testDs)
trueDimRed, predsDimRed = evaluate("cuda", modelDimRed, testDs)

print("------------------ Conv: ----------------")
classificationReport(trueConv, predsConv, data_set.encoded_dataset.label_list)
confusion_matrix(trueConv, predsConv, data_set.encoded_dataset.label_list)
print("------------------ DimRed: ----------------")
classificationReport(trueDimRed, predsDimRed, data_set.encoded_dataset.label_list)
confusion_matrix(trueDimRed, predsDimRed, data_set.encoded_dataset.label_list)


print("")

0 of 610
60 of 610
120 of 610
180 of 610
240 of 610
300 of 610
360 of 610
420 of 610
480 of 610
540 of 610
600 of 610
0 of 610
60 of 610
120 of 610
180 of 610
240 of 610
300 of 610
360 of 610
420 of 610
480 of 610
540 of 610
600 of 610
------------------ Conv: ----------------
           accuracy  precision  recall   support
   angry     0.746     0.705     0.798    99
 disgust     0.746     0.826     0.710    107
    fear     0.746     0.756     0.775    80
   happy     0.746     0.611     0.714    77
 neutral     0.746     0.787     0.779    95
     sad     0.746     0.833     0.714    91
surprise     0.746     0.710     0.721    61
                                          610

 
     avg     0.746     0.747     0.745    

                   angry     disgust        fear       happy     neutral         sad    surprise    support
       angry       0.798       0.040       0.030       0.061       0.020       0.010       0.040         99
     disgust       0.112       0.710       0.009